- https://www.kaggle.com/c/avazu-ctr-prediction
- XGBoost （特定の変数のみに絞った）
- データ数　train.csv　40428968/40428968
- 　　　　　test.csv　4577465/4577465
- submission用csvのidが桁落ち？する

In [154]:
train_kensu = 40428968
test_kensu = 4577465
kensu_str  = '40428968'
submit_csv_name = 'submit_only_certain_variables_XGBoost' + kensu_str + '.csv'

## 必要なライブラリを読み込む

In [155]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline


## データ用が多いので、絞って読み込み

In [156]:
train_fname = 'train.csv'
reader = pd.read_csv(train_fname, chunksize=train_kensu)

In [157]:
df_train = reader.get_chunk(train_kensu)

In [158]:
df_train.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [159]:
df_train.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

## ターゲット変数と特徴量を指定してXGBoostに渡せるように準備する

In [160]:
target_col = 'click'
exclude_cols = [
'click', 'id', 
'hour',
'banner_pos',
'site_id',
'site_domain',
'site_category',
'app_id',
'app_domain',
'app_category',
'device_id',
'device_ip',
'device_model',
'device_conn_type',
'C14',
'C17',
'C18',
'C19',
'C20',
'C21'
]
X = df_train.copy()
feature_cols = [col for col in X.columns if col not in exclude_cols]

In [161]:
feature_cols

['C1', 'device_type', 'C15', 'C16']

## 説明変数（特徴量）

In [162]:
X[feature_cols].head()

,C1,device_type,C15,C16
0,1005,1,320,50
1,1005,1,320,50
2,1005,1,320,50
3,1005,1,320,50
4,1005,1,320,50


## 被説明変数（ターゲット変数）

In [163]:
X[target_col].head()

0    0
1    0
2    0
3    0
4    0
Name: click, dtype: int64

In [164]:
y = np.array(X[target_col])
X = np.array(X[feature_cols])

In [165]:
y

array([0, 0, 0, ..., 0, 1, 0])

In [166]:
X

array([[1005,    1,  320,   50],
       [1005,    1,  320,   50],
       [1005,    1,  320,   50],
       ...,
       [1005,    1,  320,   50],
       [1005,    1,  320,   50],
       [1005,    1,  320,   50]])

# モデルの作成

In [167]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [168]:
X_train, X_test, y_train, y_test = \
 train_test_split(X, y, test_size=0.3, random_state=1234)

In [169]:
X_train

array([[1005,    1,  320,   50],
       [1005,    1,  320,   50],
       [1005,    1,  320,   50],
       ...,
       [1005,    1,  320,   50],
       [1005,    1,  320,   50],
       [1005,    1,  320,   50]])

## XGBoost

In [170]:
dtrain = xgb.DMatrix(X_train, label=y_train)
param = {'max_depth':3, 'learning_rate': 0.6, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(xgb.DMatrix(X_test))
print(accuracy_score(preds.round(), y_test))

[13:58:30] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[13:58:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:59:02] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
0.830139460226994


In [171]:
preds

array([0.22354136, 0.22354136, 0.22354136, ..., 0.22354136, 0.22354136,
       0.22354136], dtype=float32)

## submission用csvの作成

In [172]:
test_fname = 'test.csv'
reader_test = pd.read_csv(test_fname, chunksize=test_kensu)
df_test_origin = reader_test.get_chunk(test_kensu)

In [173]:
selected_feature_index2 = [
'C16',
'C1',
'C15',
'device_type'
]

In [174]:
df_test_origin[selected_feature_index2].head()

,C16,C1,C15,device_type
0,50,1005,320,1
1,50,1005,320,1
2,50,1005,320,1
3,50,1005,320,1
4,50,1005,320,1


In [175]:
answer = bst.predict(xgb.DMatrix(np.array(df_test_origin[selected_feature_index2]))).round().astype(int)
submit_data =  pd.Series(answer, name='click', index=df_test_origin['id'])
submit_data.to_csv(submit_csv_name, header=True)